In [ ]:
#web kazıma ile cümle, kelime ve ses dosyalarının çıkartılması ve kaydedilmesi.
path_en="html_kodları.txt"
output_path_en = "output_en.json"
import re
import json
from bs4 import BeautifulSoup

data = []

with open(path_en, "r", encoding="utf-8") as file:
    content = file.read()

for line in content.split('\n'):
    if line.startswith('#') or not line.strip():
        continue

    fields = line.split('\t')
    if len(fields) < 6:
        continue

    word = fields[2]

    html_content = fields[5].replace('""', '"')

    sound_url = ''
    if len(fields) > 6:
        sound_match = re.search(r'https://.*?\.mp3', fields[6])
        if sound_match:
            sound_url = sound_match.group(0)

    soup = BeautifulSoup(html_content, 'html.parser')
    examples = []
    for li in soup.find_all('li', class_='content-example'):
        example = li.text
        example = re.sub(r'\{\{c1::(.*?)\}\}', r'\1', example)
        examples.append(example.strip())

    data.append({
        "word": word,
        "sound_url": sound_url,
        "examples": examples
    })

with open(output_path_en, "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

print(f"Tüm veriler '{output_path}' dosyasına başarıyla kaydedildi.")


In [ ]:
#sadece ingilizce kelime ve seviyelerin çıkartılması.
import pandas as pd
df=pd.read_excel("The Oxford 5000 - excel.xlsx")
import json

level_list=[]
en_list=[]
for i in range(len(df)):
    level_list.append(df["A1 "][i])
    en_list.append(df["a "][i])
    
data=[{"Level":level,"word":en} for level,en in zip(level_list,en_list)]

output_file = "en_level.json"

with open(output_file, "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)
print(f"Veriler '{output_file}' dosyasına başarıyla kaydedildi.")

with open(output_file, 'r', encoding='utf-8') as file:
    data = json.load(file)

def temizle_bosluklar(obj):
    if isinstance(obj, str):
        return obj.strip()
    elif isinstance(obj, list):
        return [temizle_bosluklar(item) for item in obj]
    elif isinstance(obj, dict):
        return {key.strip(): temizle_bosluklar(value) for key, value in obj.items()}
    return obj

temizlenmis_data = temizle_bosluklar(data)

with open('en_level_son.json', 'w', encoding='utf-8') as file:
    json.dump(temizlenmis_data, file, ensure_ascii=False, indent=4)


In [ ]:
#cümlelerin ve kelimelerin ayrı bulunduğu dosyalardaki kelimeler eşleştirildi ve ayrı bir dosyada kelime, seviye, cümle verileri kaydedildi.

import json

with open("en_level_son.json", 'r', encoding='utf-8') as file: 
    en_level = json.load(file)
with open("output_en.json", 'r', encoding='utf-8') as file: 
    en_sentence = json.load(file)

yeni_word = []
yeni_sentence = []
yeni_level = []
buldu = 0

for i in range(len(en_level)):
    for k in range(len(en_sentence)):
        if en_level[i]["word"] == en_sentence[k]["word"]:
            buldu += 1
            yeni_word.append(en_sentence[k]["word"])
            yeni_level.append(en_level[i]["Level"])_l
            if "examples" in en_sentence[k] and en_sentence[k]["examples"]:
                yeni_sentence.append(en_sentence[k]["examples"][0])
            else:
                yeni_sentence.append("")

print(f"Eşleşen kelime sayısı: {buldu}")

data = [{"word": word, "sentence": sentence, "level": level} 
        for word, sentence, level in zip(yeni_word, yeni_sentence, yeni_level)]

unique_data = list({item['word']: item for item in data}.values())

unique_data = sorted(unique_data, key=lambda x: x['word'])

output_file = "en_sentence_level.json"
with open(output_file, "w", encoding="utf-8") as file:
    json.dump(unique_data, file, ensure_ascii=False, indent=4)

print(f"Benzersiz ve alfabetik sıralanmış veriler '{output_file}' dosyasına başarıyla kaydedildi.")


In [ ]:
#çeviri işlemleri.
import json
import os
with open("en_sentence_level.json", 'r', encoding='utf-8') as file:
    eng = json.load(file)

languages = {
    "tr": "tr_sentence_level.json",
    "de": "de_sentence_level.json",
    "fr": "fr_sentence_level.json",
    "es": "esp_sentence_level.json",
    "it": "it_sentence_level.json",
    "pt": "pr_sentence_level.json"
}

for lang, output_file in languages.items():
    translated_words = []
    translated_sentences = []
    translated_levels = []

    for en in eng:
        translated_word = GoogleTranslator(source='en', target=lang).translate(en["word"])
        translated_sentence = GoogleTranslator(source='en', target=lang).translate(en["sentence"])
 
        translated_words.append(translated_word)
        translated_sentences.append(translated_sentence)
        translated_levels.append(en["level"])

    data_translated = [
        {"word": word, "sentence": sentence, "level": level}
        for word, sentence, level in zip(translated_words, translated_sentences, translated_levels)
    ]

    with open(output_file, "w", encoding="utf-8") as file:
        json.dump(data_translated, file, ensure_ascii=False, indent=4)

    print(f"Veriler '{output_file}' dosyasına başarıyla kaydedildi.")


In [ ]:
#kelime sayısı kontrolü
import os
import json

folder_path = "C:/Users/onur_/Desktop/anki"

language_counts = {}

for file_name in os.listdir(folder_path):
    if file_name.endswith(".json"):
        language = file_name.split("_")[0]
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
            if language not in language_counts:
                language_counts[language] = 0
            language_counts[language] += len(data)

print("Her dildeki toplam veri sayıları:")
for language, count in language_counts.items():
    print(f"  {language}: {count}")

In [ ]:
#klasördeki json dosyalarını seviyelerine göre sıralama ve id ekleme
import os
import json

json_folder_path = "C:/Users/onur_/Desktop/anki"

for file_name in os.listdir(json_folder_path):
    if file_name.endswith(".json"):
        file_path = os.path.join(json_folder_path, file_name)
        
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        
        data = sorted(data, key=lambda x: x["level"])
        
        for idx, item in enumerate(data):
            item["id"] = idx+1
        
        with open(file_path, "w", encoding="utf-8") as file:
            json.dump(data, file, ensure_ascii=False, indent=4)

print("Tüm dosyalar sıralandı ve id değerleri güncellendi.")


In [ ]:
#kelimeleri indeksleme
import os
import json

json_folder_path = "C:/Users/onur_/Desktop/anki"

for file_name in os.listdir(json_folder_path):
    if file_name.endswith(".json"):
        file_path = os.path.join(json_folder_path, file_name)

        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        
        for idx, item in enumerate(data):
            item["id"] = idx + 1
        
        with open(file_path, "w", encoding="utf-8") as file:
            json.dump(data, file, ensure_ascii=False, indent=4)

print("Tüm dosyalara index eklendi.")


In [ ]:
#Kelimeleri küçük harfelere dönüştürme ve cümlelerin ilk harflerini büyük yapma.
import os
import json

json_folder_path = "C:/Users/onur_/Desktop/anki"

for file_name in os.listdir(json_folder_path):
    if file_name.endswith(".json"):
        file_path = os.path.join(json_folder_path, file_name)
        
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        
        for item in data:
            item["word"] = item["word"].lower()
            
            if item.get("sentence"):
                item["sentence"] = item["sentence"].capitalize()
        with open(file_path, "w", encoding="utf-8") as file:
            json.dump(data, file, ensure_ascii=False, indent=4)

print("Tüm dosyalar güncellendi.")
